In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
zip_file = 'drive/MyDrive/AI/CNN/numbers/Numbers_Images_Dataset.zip'
with zipfile.ZipFile(zip_file,'r') as zip_ref:
    zip_ref.extractall('images')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
directory_path = "/content/images"
images_df = pd.read_csv(directory_path + "/numbers.csv")
images_df

,origin,group,label,file
0,chars74k,GoodImg,8,chars74k_png/GoodImg/Sample8/img009-00029.png
1,chars74k,GoodImg,8,chars74k_png/GoodImg/Sample8/img009-00032.png
2,chars74k,GoodImg,8,chars74k_png/GoodImg/Sample8/img009-00020.png
3,chars74k,GoodImg,8,chars74k_png/GoodImg/Sample8/img009-00027.png
4,chars74k,GoodImg,8,chars74k_png/GoodImg/Sample8/img009-00022.png
...,...,...,...,...
71585,mnist,Hnd,0,mnist_png/Hnd/Sample0/45577.png
71586,mnist,Hnd,0,mnist_png/Hnd/Sample0/19491.png
71587,mnist,Hnd,0,mnist_png/Hnd/Sample0/30425.png
71588,mnist,Hnd,0,mnist_png/Hnd/Sample0/11240.png


In [ ]:
from pathlib import Path

def create_directory(directory_path):
    # Create a directory if it doesn't exist
    try:
        Path(directory_path).mkdir(parents=True, exist_ok=True)
        print(f"Directory '{directory_path}' created successfully.")
    except OSError as e:
        print(f"Error creating directory: {e}")

import shutil

def copy_file(source_path, destination_path):
    try:
        shutil.copy(source_path, destination_path)
        print(f"File copied from '{source_path}' to '{destination_path}' successfully.")
    except Exception as e:
        print(f"Error copying file: {e}")

In [ ]:
create_directory(directory_path + "/" + "test")
create_directory(directory_path + "/" + "train")
folders = []
for i in range(0,10):
    folders.append(f"Sample{i}")
    create_directory(f'{directory_path}/train/Sample{i}')
    create_directory(f'{directory_path}/test/Sample{i}')

Directory '/content/images/test' created successfully.
Directory '/content/images/train' created successfully.
Directory '/content/images/train/Sample0' created successfully.
Directory '/content/images/test/Sample0' created successfully.
Directory '/content/images/train/Sample1' created successfully.
Directory '/content/images/test/Sample1' created successfully.
Directory '/content/images/train/Sample2' created successfully.
Directory '/content/images/test/Sample2' created successfully.
Directory '/content/images/train/Sample3' created successfully.
Directory '/content/images/test/Sample3' created successfully.
Directory '/content/images/train/Sample4' created successfully.
Directory '/content/images/test/Sample4' created successfully.
Directory '/content/images/train/Sample5' created successfully.
Directory '/content/images/test/Sample5' created successfully.
Directory '/content/images/train/Sample6' created successfully.
Directory '/content/images/test/Sample6' created successfully.


In [ ]:
for folder in folders:
    images_df['file'] = images_df['file'].astype('string')
    df = images_df[images_df['file'].str.contains(folder,case=False,na=False)]
    five_percent = int(df.shape[0]/20)
    for i in range(0,five_percent):
        source = directory_path + "/numbers/" + df['file'].iloc[i]
        dest = directory_path + "/"  + "test" + "/" + folder
        copy_file(source,dest)

    for i in range(five_percent,int(len(df)/2)):
        source = directory_path + "/numbers/" + df['file'].iloc[i]
        dest = directory_path + "/"  + "train" + "/" + folder
        copy_file(source,dest)

In [ ]:
tf.config.set_visible_devices([],'GPU')
print('Available GPU cores ', len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_genarator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    )

test_data_generator = ImageDataGenerator(
    rescale=1./255
    )

In [ ]:
train_data = train_data_genarator.flow_from_directory(
    directory_path + "/" + 'train',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_data_generator.flow_from_directory(
        directory_path + "/" + 'test',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
cnn = tf.keras.models.Sequential()
# First convolutional layer

cnn.add(tf.keras.layers.Conv2D(
          filters=64,
          kernel_size=3,
          activation='relu',
          input_shape=[64,64,3]
          )
)

cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='valid'
    )
)

# hidden convolutional layer

cnn.add(tf.keras.layers.Conv2D(
          filters=64,
          kernel_size=3,
          activation='relu'
          )
)

cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2,2),
    strides=(2,2),
    padding='valid'
    )
)



In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(64,activation='relu'))
# Output layer
cnn.add(tf.keras.layers.Dense(10,activation='relu'))

In [ ]:
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
cnn.fit(train_data,validation_data=test_data,epochs=10,batch_size=32)

In [ ]:
directory_path = "/content/drive/MyDrive/AI/CNN/numbers/"
from joblib import dump
dump(cnn, directory_path + 'numbers.joblib')

['/content/drive/MyDrive/AI/CNN/numbers/numbers.joblib']

In [ ]:
from joblib import load
import os
from tensorflow.keras.preprocessing import image

directory_path = "/content/drive/MyDrive/AI/CNN/numbers/"
cnn = load(directory_path + 'numbers.joblib')

directory_path = "/content/drive/MyDrive/AI/CNN/numbers/new_data"
images = [f for f in os.listdir(directory_path)]

results = []
for my_image in images:
    print(my_image)
    test_img = image.load_img(directory_path + "/" + my_image,target_size=(64,64))
    test_image = image.img_to_array(test_img)
    test_image = test_image/255.0
    test_image = np.expand_dims(test_image,axis=0)
    result = cnn.predict(test_image)
    results.append(np.argmax(result))
    print(result)
print(results)

1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
[[  0.       438.32736   91.19762  198.85774  139.30815   89.03803
  155.37486   45.837624  81.78064  168.6587  ]]
2.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[  0.          0.        227.65346   162.63617     0.         59.300488
    0.          0.          0.          1.1975707]]
3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[  0.       429.30554   84.45976  222.82478  100.606094  91.268265
  153.86354   83.67351   81.59422  178.89612 ]]
4.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[  0.       436.87872   89.728836 196.31572  144.15724   92.052765
  159.38673   39.618637  81.597374 166.21756 ]]
5.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[  0.       435.01996   90.143005 202.31734  137.09753   90.012115
  155.4859    51.368156  81.39887  170.32204 ]]
6.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[  0.        0.        0.      444.00967   0.      169.48727 222.31639
    0.       40.55761 239.6561 ]]
7.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/st